# Restaurants at New York City newly build houses.

## Contents
- Introduction
- Data cleaning
- Data exploration
- Hypothesis testing
- Visualization

The dataset contains data about permits for construction and demolition activities in the City of New York. These permatis are issued for maintenance, plumbing, demolition and other types of work. I will use the information about buildings' age to decide if there is a link between the age and a quality of a restaurant. For this project I will use another [dataset](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j), which provides information about healt inspections at New York restaurants.

The datasets are large enough to be stored in a database, so I copied it to PostreSQL.
The permit dataset has 60 columns, but I will use only addresses, permit types and permit dates. 
I will start from cleaning the data. While addresses and streets can be difficult to validate, I can easily do this with dates, zip codes and borough.

In [7]:
from sqlalchemy import create_engine
import zipcodes

In [128]:
engine = create_engine('postgresql://sofiagodovykh:@localhost/DS')
connection = engine.connect()
borough = connection.execute('SELECT DISTINCT(borough) FROM nyc;')
for item in borough:
    print(item)

('BRONX',)
('MANHATTAN',)
('BROOKLYN',)
('QUEENS',)
('STATEN ISLAND',)


Bogough column is fine. Then I will check zipcodes, using zipcodes library.

In [57]:
addresses = set()
problems = []
sql_zips = connection.execute('SELECT DISTINCT(zipcode) FROM nyc;')
for zip in sql_zips:
    try:
        temp = zipcodes.matching(str(zip[0]))
        addresses.add(temp[0]['city'] + ' ' + temp[0]['state'])
    except:
        problems.append(zip[0])
    
print(addresses)
print(problems)

{'RIDGEWOOD NY', 'CORONA NY', 'BRONX NY', 'BAYSIDE NY', 'MASPETH NY', 'LITTLE NECK NY', 'CAMBRIA HEIGHTS NY', 'FOREST HILLS NY', 'ASTORIA NY', 'ELMHURST NY', 'COLLEGE POINT NY', 'SUNNYSIDE NY', 'SPRINGFIELD GARDENS NY', 'JAMAICA NY', 'ROSEDALE NY', 'RICHMOND HILL NY', 'EAST ELMHURST NY', 'BLOOMING GROVE NY', 'FLUSHING NY', 'HOWARD BEACH NY', 'GLEN OAKS NY', 'OZONE PARK NY', 'HOLLIS NY', 'REGO PARK NY', 'KEW GARDENS NY', 'NEW HYDE PARK NY', 'SOUTH RICHMOND HILL NY', 'SOUTH OZONE PARK NY', 'WOODSIDE NY', 'BROOKLYN NY', 'MIDDLE VILLAGE NY', 'CHESTER NY', 'QUEENS VILLAGE NY', 'OAKLAND GARDENS NY', 'FAR ROCKAWAY NY', 'JACKSON HEIGHTS NY', 'WHITESTONE NY', 'SAINT ALBANS NY', 'BREEZY POINT NY', 'ARVERNE NY', 'FLORAL PARK NY', 'WOODHAVEN NY', 'LONG ISLAND CITY NY', 'ROCKAWAY PARK NY', 'FRESH MEADOWS NY', 'BELLEROSE NY', 'NEW YORK NY', 'STATEN ISLAND NY'}
[None, 10000, 4452, 10487, 0, 11376]


Most of the zipcodes are in New York metropolitan area. Let's see how many of them a erroneous.

In [73]:
connection.execute('SELECT COUNT(zipcode) FROM nyc WHERE zipcode IN (NULL, 10000, 4452, 10487, 0, 11376);').scalar()

246

There is only 246 wrong zipcodes. I will delete them.

In [74]:
connection.execute('DELETE FROM nyc WHERE zipcode IN (NULL, 10000, 4452, 10487, 0, 11376);')

Then I will take care of dates. I am mostly interested in expirationdate, since it is an indicator that building is finally finished. I have ran a query to find the earliest and the latest dates of permits for building new houses.

In [126]:
print(connection.execute('SELECT DISTINCT(expirationdate) FROM nyc WHERE expirationdate = '
                   '(SELECT MAX(expirationdate) from nyc WHERE permittype = \'NB\');').scalar())
print(connection.execute('SELECT DISTINCT(expirationdate) FROM nyc WHERE expirationdate = '
                   '(SELECT MIN(expirationdate) from nyc WHERE permittype = \'NB\');').scalar())

2020-04-01


1990-02-01


This data looks valid. But there is a tricky part: permits can be reissued, so it is possible that building has multiple permits. For my purpose, I need to use only the final one.

In [129]:
connection.execute('SELECT COUNT(DISTINCT(CONCAT(borough, zipcode, house, streetname))) FROM nyc WHERE permittype = \'NB\';').scalar()

86141

So, the dataset stores an information about 86141 houses which were built since 1990. I am very satisfied with this dataset. The next step is to validate another one.